# Create Embedding
In this notebook, we'll connect to a Neo4j instance.  We'll load data and compute an embedding.  The notebook exports that data to pandas and then CSV files.

## Using the Neo4j API
Let's connect to our Neo4j deployment.  First off, install the Neo4j Graph Data Science package.

In [1]:
%pip install graphdatascience

     |████████████████████████████████| 182 kB 6.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


Now, you're going to need the connection string and credentials from the deployment you created above.

In [4]:
# Edit these variables!
DB_URL = 'neo4j://vm0.node-abcd.southeastasia.cloudapp.azure.com:7687'
DB_PASS = 'mypassword'

# You can leave this default
DB_USER = 'neo4j'

In [5]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(DB_URL, auth=(DB_USER, DB_PASS))

## Graph Data Science
Now we're going to use Neo4j Graph Data Science to create an in memory graph represtation of the data.  We'll enhance that represation with features we engineer using a graph embedding.

In [7]:
result = gds.run_cypher(
  """
    CALL gds.graph.project(
      'mygraph',
      ['Company', 'Manager', 'Holding'],
      {
          OWNS: {orientation: 'UNDIRECTED'},
          PARTOF: {orientation: 'UNDIRECTED'}
      }
    )
    YIELD
      graphName AS graph,
      relationshipProjection AS readProjection,
      nodeCount AS nodes,
      relationshipCount AS rels
  """
)
display(result)

,graph,readProjection,nodes,rels
0,mygraph,"{'PARTOF': {'orientation': 'UNDIRECTED', 'aggr...",458170,1787688


If you get an error saying the graph already exists, that's probably because you ran this code before. You can destroy it using this command:

In [6]:
result = gds.run_cypher(
  """
    CALL gds.graph.drop('mygraph')
  """
)
display(result)

,graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema
0,mygraph,neo4j,,-1,458170,1787688,{'relationshipProjection': {'PARTOF': {'orient...,0.000009,2022-10-20T08:07:29.014650000+00:00,2022-10-20T08:25:10.719504000+00:00,"{'graphProperties': {}, 'relationships': {'PAR..."


Now, let's list the details of the graph to make sure the projection was created as we want.

In [8]:
result = gds.run_cypher(
  """
    CALL gds.graph.list()
  """
)
display(result)

,degreeDistribution,graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema
0,"{'p99': 18, 'min': 1, 'max': 6864, 'mean': 3.9...",mygraph,neo4j,25 MiB,26961560,458170,1787688,{'relationshipProjection': {'PARTOF': {'orient...,0.000009,2022-10-21T04:15:38.574979000+00:00,2022-10-21T04:15:38.773557000+00:00,"{'graphProperties': {}, 'relationships': {'PAR..."


Now we can generate an embedding from that graph. This is a new feature we can use in our predictions. We're using FastRP, which is a more full featured and higher performance of Node2Vec. You can learn more about that [here](https://neo4j.com/docs/graph-data-science/current/algorithms/fastrp/).

There are a bunch of parameters we could adjust in this.  One of the most obvious is the embeddingDimension.  The documentation covers many more.

In [9]:
result = gds.run_cypher(
  """
  CALL gds.fastRP.mutate('mygraph',{
    embeddingDimension: 16,
    randomSeed: 1,
    mutateProperty:'embedding'
  })
  """
)
display(result)

,nodePropertiesWritten,mutateMillis,nodeCount,preProcessingMillis,computeMillis,configuration
0,458170,0,458170,0,274,"{'nodeSelfInfluence': 0, 'relationshipWeightPr..."


That creates an embedding for each node type.  However, we only want the embedding on the nodes of type holding.

We're going to take the embedding from our projection and write it to the holding nodes in the underlying database.

In [10]:
result = gds.run_cypher(
  """
    CALL gds.graph.writeNodeProperties('mygraph', ['embedding'], ['Holding'])
    YIELD writeMillis
  """
)
display(result)

,writeMillis
0,788


In [11]:
result = gds.run_cypher(
  """
    MATCH (n:Holding) RETURN n
  """
)
display(result)

,n
0,"(shares, cusip, reportCalendarOrQuarter, filin..."
1,"(shares, cusip, reportCalendarOrQuarter, filin..."
2,"(shares, cusip, reportCalendarOrQuarter, filin..."
3,"(shares, cusip, reportCalendarOrQuarter, filin..."
4,"(shares, cusip, reportCalendarOrQuarter, filin..."
...,...
446917,"(shares, cusip, reportCalendarOrQuarter, filin..."
446918,"(shares, cusip, reportCalendarOrQuarter, filin..."
446919,"(shares, cusip, reportCalendarOrQuarter, filin..."
446920,"(shares, cusip, reportCalendarOrQuarter, filin..."


Note that this query will take 2-3 minutes to run as it's grabbing nearly half a million nodes along with all their properties and our new embedding.

## Pandas
Now we're going to reformat the query output.

In [12]:
import pandas as pd
df = pd.DataFrame([dict(record.items()) for record in result['n']])
df

,shares,cusip,reportCalendarOrQuarter,filingManager,embedding,nameOfIssuer,value,target
0,270,88579Y101,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,"[-0.06891358643770218, 0.1725142002105713, -0....",3M Co,52024000,False
1,195,00508Y102,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,"[-0.0012208839179947972, 0.005601316690444946,...",Acuity Brands Inc,32175000,False
2,4939,00724F101,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,"[0.11818736791610718, 0.26252642273902893, -0....",Adobe Systems Inc,2347852000,False
3,1557,02079K305,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,"[0.14201897382736206, 0.2642807066440582, 0.14...",Alphabet Inc A,3211344000,False
4,837,02079K107,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,"[0.30085813999176025, 0.2475065439939499, 0.44...",Alphabet Inc C,1731443000,False
...,...,...,...,...,...,...,...,...
446917,56874,911312106,09-30-2021,LEE DANNER & BASS INC,"[-0.10755999386310577, 0.09883543848991394, 0....",United Parcel Svc. Cl B,10357000,False
446918,231000,92552v100,09-30-2021,LEE DANNER & BASS INC,"[-0.19772684574127197, 0.19481556117534637, 0....",ViaSat Inc,12721000,True
446919,55104,92826C839,09-30-2021,LEE DANNER & BASS INC,"[0.19159139692783356, 0.5284039974212646, 0.15...",Visa Inc,12274000,True
446920,79459,931142103,09-30-2021,LEE DANNER & BASS INC,"[0.16196903586387634, 0.5445767045021057, 0.60...",Wal-Mart Stores Inc.,11075000,False


Note that the embedding row is an array. To make this dataset more consumable, we should flatten that out into multiple individual features: embedding_0, embedding_1, ... embedding_n.


In [13]:
embeddings = pd.DataFrame(df['embedding'].values.tolist()).add_prefix("embedding_")
merged = df.drop(columns=['embedding']).merge(embeddings, left_index=True, right_index=True)
merged

,shares,cusip,reportCalendarOrQuarter,filingManager,nameOfIssuer,value,target,embedding_0,embedding_1,embedding_2,...,embedding_6,embedding_7,embedding_8,embedding_9,embedding_10,embedding_11,embedding_12,embedding_13,embedding_14,embedding_15
0,270,88579Y101,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,3M Co,52024000,False,-0.068914,0.172514,-0.270718,...,0.142907,0.707541,0.255056,0.168236,0.014960,-0.183164,0.214030,-0.149799,0.436635,-0.393186
1,195,00508Y102,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,Acuity Brands Inc,32175000,False,-0.001221,0.005601,-0.008561,...,-0.249999,-0.027272,0.210765,0.232817,0.275897,-0.276700,0.338566,-0.479243,0.789944,-0.336510
2,4939,00724F101,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,Adobe Systems Inc,2347852000,False,0.118187,0.262526,-0.081295,...,-0.591563,0.398565,0.239687,-0.259427,-0.352596,0.025593,0.626399,0.247615,0.445566,0.094596
3,1557,02079K305,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,Alphabet Inc A,3211344000,False,0.142019,0.264281,0.142079,...,-0.150988,1.035650,0.370033,-0.064532,-0.239375,-0.003771,0.124420,0.027277,0.077211,-0.477533
4,837,02079K107,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,Alphabet Inc C,1731443000,False,0.300858,0.247507,0.449530,...,-0.415947,0.365203,-0.428251,-0.000808,0.304490,0.002407,-0.100384,-0.317436,0.368293,-0.037012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446917,56874,911312106,09-30-2021,LEE DANNER & BASS INC,United Parcel Svc. Cl B,10357000,False,-0.107560,0.098835,0.136811,...,-0.246284,0.304195,0.127205,-0.927462,-0.280658,0.211018,-0.149063,-0.060732,-0.073806,-0.089820
446918,231000,92552v100,09-30-2021,LEE DANNER & BASS INC,ViaSat Inc,12721000,True,-0.197727,0.194816,0.100359,...,-0.246779,0.370052,0.573178,0.061018,-0.038240,-0.423521,-0.037658,-0.372200,-0.453317,-0.374625
446919,55104,92826C839,09-30-2021,LEE DANNER & BASS INC,Visa Inc,12274000,True,0.191591,0.528404,0.158589,...,-0.766427,-0.110386,0.576788,-0.022191,-0.768111,-0.192285,-0.119118,0.234878,0.124492,-0.344153
446920,79459,931142103,09-30-2021,LEE DANNER & BASS INC,Wal-Mart Stores Inc.,11075000,False,0.161969,0.544577,0.602678,...,-0.070697,-0.004854,0.217884,-0.519946,-0.045873,-0.047081,-0.103562,0.011971,-0.375597,-0.362491


Now that we have the data formatted properly, let's split it into training, testing and validation sets.  We'll write those to disk.

In [14]:
import os 

df = merged

outdir = './data'
if not os.path.exists(outdir):
    os.mkdir(outdir)

training_dir = './data/training-mltable-folder'
if not os.path.exists(training_dir):
    os.mkdir(training_dir)

train = df.loc[df['reportCalendarOrQuarter'] == '03-31-2021']
train.to_csv(os.path.join(training_dir, 'train.csv'), index=False)

test_dir = './data/test-mltable-folder'
if not os.path.exists(test_dir):
    os.mkdir(test_dir)

test = df.loc[df['reportCalendarOrQuarter'] == '06-30-2021']
test = test.drop(['target'], axis=1)
test.to_csv(os.path.join(test_dir, 'test.csv'), index=False)

validate_dir = './data/validation-mltable-folder'
if not os.path.exists(validate_dir):
    os.mkdir(validate_dir)

validate = df.loc[df['reportCalendarOrQuarter'] == '09-30-2021']
validate = validate.drop(['target'], axis=1)
validate.to_csv(os.path.join(validate_dir, 'validate.csv'), index=False)